In [134]:
import pandas as pd

url = "postgresql://localhost/test?user=postgres&password=248655"


def batch_load_sql(query: str) -> pd.DataFrame:
    CHUNKSIZE = 200000

    chunks = []
    for chunk_dataframe in pd.read_sql(query, url, chunksize=CHUNKSIZE):
        chunks.append(chunk_dataframe)
    return pd.concat(chunks, ignore_index=True)


data = batch_load_sql("""
SELECT co.visitor_token_id, 
	ss.title,
	sa.answer_text, sa.question_id,
	sq.question_text
FROM core_order co
JOIN core_orderitem coi ON coi.order_id = co.id
JOIN core_ticket ct ON ct.id=coi.ticket_id
JOIN core_tickettype ctt ON ctt.id=ct.ticket_type_id
JOIN surveys_surveyresult ssr ON ssr.id=ct.survey_result_id
JOIN surveys_survey ss ON ss.id=ssr.survey_id
JOIN surveys_answer sa ON ssr.id=sa.survey_result_id
JOIN surveys_question sq ON sq.id=sa.question_id
WHERE co.visitor_token_id != '' AND co.status=3 AND ctt.is_event=False AND co.shop_id=4""")

In [135]:
data.head()

,visitor_token_id,title,answer_text,question_id,question_text
0,f0fbc90e94ee756fbb26f2cc2efb579ca1a306b3,Анкета посетителя ФРР 2023,Бар,6,Предприятия питания и торговли
1,f0fbc90e94ee756fbb26f2cc2efb579ca1a306b3,Анкета посетителя ФРР 2023,Городской отель,7,Гостиницы
2,f0fbc90e94ee756fbb26f2cc2efb579ca1a306b3,Анкета посетителя ФРР 2023,Продукты питания и напитки,8,Производители
3,f0fbc90e94ee756fbb26f2cc2efb579ca1a306b3,Анкета посетителя ФРР 2023,"Ассоциации, государственные структуры",9,Поставщики
4,2575afcb2bbe2d0df5e374a20f1f2f2c4bf2368c,Анкета посетителя ФРР 2023,Бар,6,Предприятия питания и торговли


In [136]:
data.title.value_counts()

title
Посетитель ПИР ЭКСПО 2024     368807
Анкета посетителя 2022         47120
Анкета посетителя ФРР 2023       309
Name: count, dtype: int64

In [137]:
data[['question_id', 'question_text']].value_counts()

question_id  question_text                                                                                                 
20           Название компании                                                                                                 30905
12           Фамилия                                                                                                           30905
13           Имя                                                                                                               30905
14           Адрес электронной почты                                                                                           30905
15           Город                                                                                                             30905
17           Должность                                                                                                         30905
18           Мобильный телефон                                                

In [138]:
questions_2drop = [11, 12, 13, 14, 15, 18, 19, 20, 25, 26]

data = data[(~data['question_id'].isin(questions_2drop))]

In [139]:
data

,visitor_token_id,title,answer_text,question_id,question_text
0,f0fbc90e94ee756fbb26f2cc2efb579ca1a306b3,Анкета посетителя ФРР 2023,Бар,6,Предприятия питания и торговли
1,f0fbc90e94ee756fbb26f2cc2efb579ca1a306b3,Анкета посетителя ФРР 2023,Городской отель,7,Гостиницы
2,f0fbc90e94ee756fbb26f2cc2efb579ca1a306b3,Анкета посетителя ФРР 2023,Продукты питания и напитки,8,Производители
3,f0fbc90e94ee756fbb26f2cc2efb579ca1a306b3,Анкета посетителя ФРР 2023,"Ассоциации, государственные структуры",9,Поставщики
4,2575afcb2bbe2d0df5e374a20f1f2f2c4bf2368c,Анкета посетителя ФРР 2023,Бар,6,Предприятия питания и торговли
...,...,...,...,...,...
416231,c2b71aaa54361b30fbb38176928a2d7182b59cd1,Посетитель ПИР ЭКСПО 2024,True,36,Городской отель
416232,c2b71aaa54361b30fbb38176928a2d7182b59cd1,Посетитель ПИР ЭКСПО 2024,True,43,Общение с коллегами / Установление новых делов...
416233,c2b71aaa54361b30fbb38176928a2d7182b59cd1,Посетитель ПИР ЭКСПО 2024,True,44,"Изучение рынка, знакомство с новинками / Получ..."
416234,c2b71aaa54361b30fbb38176928a2d7182b59cd1,Посетитель ПИР ЭКСПО 2024,True,46,Образовательные цели/ Посещение деловых меропр...


In [140]:
pivot = data.pivot_table(
    index='visitor_token_id',
    columns='question_text',
    values='answer_text',
    aggfunc=lambda x: ' '.join(map(str, x))
).reset_index()

pivot

question_text,visitor_token_id,Dark Kitchen. Доставка еды.,Streat food,"Ассоциации, государственные структуры",Бар,Глэмпинги и средства размещения с быстровозводимыми конструкциями,Городской отель,Гостиницы,Должность,Загородный отель/средство размещения,...,Предприятия питания и торговли,Продвижение собственных товаров и услуг,"Продуктовый ритейл (гипермаркет, супермаркет и пр.)",Продукты питания и напитки,Производители,Ресторан/кафе,Самозанятый кондитер,Санаторно-оздоровительное средство размещения,Сфера деятельности,Управляющая компания
0,0000b08340cc86c7a61cc3251fbf2544c6472dda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Бариста, наставник",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0004214be6142004e5dfae0c869f7648c33e3640,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Менеджер по работе с ключевыми клиентами,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00099e4722876bce6828a4d083f59d9bc80284d1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Ресторан/кафе,NaN,NaN,NaN,Продукты питания и напитки,NaN,NaN,NaN,NaN,NaN
3,000aa3c29eee38748aa67148ec80f8754f27017f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Кулинарная студия,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,000cf2d412eb10cae69abe61b606e8be3063d9ba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Ресторан/кафе Ресторан/кафе,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38644,fff47c137b2ba6a66e4670fe3701f3088682b86e,NaN,NaN,NaN,NaN,NaN,True,NaN,Су шеф кондитер,NaN,...,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38645,fff5ecf120366e7a9982e7e01666f84280478336,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38646,fff6302be5388f9160e86cb11a6be2cf487a1c5b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Founder,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38647,fff676d8522543716287e474ed6eebe50db360ab,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Генеральный директор,NaN,...,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN


In [141]:
pivot_cols = pivot.drop(columns=['visitor_token_id', 'Должность'], axis=1).columns.to_list()
pivot_cols

['Dark Kitchen. Доставка еды.',
 'Streat food',
 'Ассоциации, государственные структуры',
 'Бар',
 'Глэмпинги и средства размещения с быстровозводимыми конструкциями',
 'Городской отель',
 'Гостиницы',
 'Загородный отель/средство размещения',
 'Зеленое и обжаренное зерно',
 'Изучение рынка, знакомство с новинками / Получение актуальной информации о тенденциях рынка, конкурентах и пр.',
 'Интересующие темы деловой программы',
 'Кондитерская / Пекарня',
 'Кофейня',
 'Курортный отель/средство размещения',
 'Малое средство размещения (хостел, бутик-отель, коливинг, и др.)',
 'Оборудование и услуги для гостиниц',
 'Оборудование и услуги для ресторанов и продуктового ритейла',
 'Образовательные цели/ Посещение деловых мероприятий',
 'Общение с коллегами / Установление новых деловых контактов',
 'Пиццерия',
 'Поиск поставщиков и сбор информации для будущих закупок / Поиск продукции или услуг для бизнеса',
 'Поставщики',
 'Предприятия питания и торговли',
 'Продвижение собственных товаров и ус

In [142]:
import re


def clean_text(x):
    if pd.notna(x):
        s = str(x).strip().lower()

        s = re.sub(r'(?<=\s)[^\w\s]+(?=\s)', '', s)
        s = re.sub(r'(?<=\w|\d)[\W\d]+(?=\w|\d)', ' ', s)
        s = re.sub(r'(?<=\w|\d)[\W\d]+$', ' ', s)
        s = re.sub(r'^[^a-zA-Zа-яА-Я]+', '', s)

        return ' '.join(s.strip().split())
    return ''


def job_preprocess(x):
    return ' '.join(set(x.split()))


user_data = pd.DataFrame()
user_data['visitor_token_id'] = pivot['visitor_token_id']

user_data['job'] = pivot['Должность'].apply(clean_text)
# user_data['job'] = user_data['job'].apply(job_preprocess) - плохо для BERT
user_data

,visitor_token_id,job
0,0000b08340cc86c7a61cc3251fbf2544c6472dda,бариста наставник
1,0004214be6142004e5dfae0c869f7648c33e3640,менеджер по работе с ключевыми клиентами
2,00099e4722876bce6828a4d083f59d9bc80284d1,
3,000aa3c29eee38748aa67148ec80f8754f27017f,
4,000cf2d412eb10cae69abe61b606e8be3063d9ba,
...,...,...
38644,fff47c137b2ba6a66e4670fe3701f3088682b86e,су шеф кондитер
38645,fff5ecf120366e7a9982e7e01666f84280478336,
38646,fff6302be5388f9160e86cb11a6be2cf487a1c5b,founder
38647,fff676d8522543716287e474ed6eebe50db360ab,генеральный директор


In [143]:
# import spacy
# 
# nlp = spacy.load("ru_core_news_sm")
# user_data['job'] = user_data['job'].apply(lambda x: ' '.join([token.lemma_ for token in nlp(x)]))

In [146]:
pivot

question_text,visitor_token_id,Dark Kitchen. Доставка еды.,Streat food,"Ассоциации, государственные структуры",Бар,Глэмпинги и средства размещения с быстровозводимыми конструкциями,Городской отель,Гостиницы,Должность,Загородный отель/средство размещения,...,Предприятия питания и торговли,Продвижение собственных товаров и услуг,"Продуктовый ритейл (гипермаркет, супермаркет и пр.)",Продукты питания и напитки,Производители,Ресторан/кафе,Самозанятый кондитер,Санаторно-оздоровительное средство размещения,Сфера деятельности,Управляющая компания
0,0000b08340cc86c7a61cc3251fbf2544c6472dda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Бариста, наставник",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0004214be6142004e5dfae0c869f7648c33e3640,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Менеджер по работе с ключевыми клиентами,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00099e4722876bce6828a4d083f59d9bc80284d1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Ресторан/кафе,NaN,NaN,NaN,Продукты питания и напитки,NaN,NaN,NaN,NaN,NaN
3,000aa3c29eee38748aa67148ec80f8754f27017f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Кулинарная студия,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,000cf2d412eb10cae69abe61b606e8be3063d9ba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Ресторан/кафе Ресторан/кафе,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38644,fff47c137b2ba6a66e4670fe3701f3088682b86e,NaN,NaN,NaN,NaN,NaN,True,NaN,Су шеф кондитер,NaN,...,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38645,fff5ecf120366e7a9982e7e01666f84280478336,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38646,fff6302be5388f9160e86cb11a6be2cf487a1c5b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Founder,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38647,fff676d8522543716287e474ed6eebe50db360ab,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Генеральный директор,NaN,...,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN


In [151]:
def params_preprocess(row):
    words = []
    for col in pivot_cols:
        if pd.notna(row[col]) and row[col] != '':
            words.append([col.lower()] if 'true' in str(row[col]).lower() else str(row[col]))

    return ' '.join(map(clean_text, words))


user_data['params'] = pivot[pivot_cols].apply(params_preprocess, axis=1)
user_data

,visitor_token_id,job,params
0,0000b08340cc86c7a61cc3251fbf2544c6472dda,бариста наставник,
1,0004214be6142004e5dfae0c869f7648c33e3640,менеджер по работе с ключевыми клиентами,
2,00099e4722876bce6828a4d083f59d9bc80284d1,,ресторан кафе продукты питания и напитки
3,000aa3c29eee38748aa67148ec80f8754f27017f,,кулинарная студия
4,000cf2d412eb10cae69abe61b606e8be3063d9ba,,ресторан кафе ресторан кафе
...,...,...,...
38644,fff47c137b2ba6a66e4670fe3701f3088682b86e,су шеф кондитер,городской отель изучение рынка знакомство с но...
38645,fff5ecf120366e7a9982e7e01666f84280478336,,
38646,fff6302be5388f9160e86cb11a6be2cf487a1c5b,founder,кофейня общение с коллегами установление новых...
38647,fff676d8522543716287e474ed6eebe50db360ab,генеральный директор,изучение рынка знакомство с новинками получени...


In [152]:
# user_data['params'] = user_data['params'].apply(lambda x: ' '.join([token.lemma_ for token in nlp(x)]))

In [153]:
user_data.to_csv('Data/users.csv', index=False)

In [154]:
import nltk
from nltk.corpus import stopwords
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('stopwords')
russian_stopwords = list(set(stopwords.words('russian')))
english_stopwords = list(set(stopwords.words('english')))


class TfidfClusterTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.vectorizer = TfidfVectorizer(stop_words=russian_stopwords + english_stopwords)
        self.pca = PCA(n_components=20)
        self.kmeans = KMeans(n_clusters=5)
        self.scaler = StandardScaler()

    def fit(self, X, y=None):
        tfidf_matrix = self.vectorizer.fit_transform(raw_documents=X)
        pca_res = self.pca.fit_transform(tfidf_matrix)
        self.scaler.fit(self.kmeans.fit_transform(pca_res))
        return self

    def transform(self, X):
        tfidf_matrix = self.vectorizer.transform(X)
        res = self.kmeans.transform(self.pca.transform(tfidf_matrix))
        return pd.DataFrame(self.scaler.transform(res))

    @staticmethod
    def get_feature_names_out(self, input_features):
        return ['d2c1', 'd2c2', 'd2c3', 'd2c4', 'd2c5']

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\merku\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [158]:
from sklearn.compose import ColumnTransformer

t = [
    ('job', TfidfClusterTransformer(), 'job'),
    ('par', TfidfClusterTransformer(), 'params'),
]

col_transformer = ColumnTransformer(transformers=t,
                                    remainder='passthrough',
                                    force_int_remainder_cols=False)

In [159]:
model_data = col_transformer.fit_transform(user_data)

cols = pd.Series(col_transformer.get_feature_names_out()).apply(lambda x: x.replace('__', '_'))
cols[10] = 'visitor_token_id'

model_data = pd.DataFrame(model_data, columns=cols)
model_data.to_csv('Data/users_tfidf.csv', index=False)
model_data

,job_d2c1,job_d2c2,job_d2c3,job_d2c4,job_d2c5,par_d2c1,par_d2c2,par_d2c3,par_d2c4,par_d2c5,visitor_token_id
0,0.185688,-2.045327,0.114999,0.155288,-0.050801,-2.324487,-0.885032,-0.396823,-0.563423,-1.237862,0000b08340cc86c7a61cc3251fbf2544c6472dda
1,-0.272121,-0.138362,-1.521077,-0.204269,-0.356814,-2.324487,-0.885032,-0.396823,-0.563423,-1.237862,0004214be6142004e5dfae0c869f7648c33e3640
2,-0.801873,-0.299506,-0.413816,-0.414564,-0.551328,0.644484,-0.670613,1.064075,0.563896,-2.574745,00099e4722876bce6828a4d083f59d9bc80284d1
3,-0.801873,-0.299506,-0.413816,-0.414564,-0.551328,-2.312398,-0.879836,-0.390965,-0.559752,-1.231558,000aa3c29eee38748aa67148ec80f8754f27017f
4,-0.801873,-0.299506,-0.413816,-0.414564,-0.551328,0.637404,-3.152206,1.043217,0.562031,0.622847,000cf2d412eb10cae69abe61b606e8be3063d9ba
...,...,...,...,...,...,...,...,...,...,...,...
38644,0.971251,0.736872,0.978918,1.066409,0.84557,0.420536,0.60255,-1.378484,0.563236,0.459221,fff47c137b2ba6a66e4670fe3701f3088682b86e
38645,-0.801873,-0.299506,-0.413816,-0.414564,-0.551328,-2.324487,-0.885032,-0.396823,-0.563423,-1.237862,fff5ecf120366e7a9982e7e01666f84280478336
38646,-0.79129,-0.294199,-0.407777,-0.407524,-0.544259,0.492349,0.605744,-0.402559,0.060447,0.512933,fff6302be5388f9160e86cb11a6be2cf487a1c5b
38647,1.528695,1.406708,1.577733,-0.573587,1.472958,0.440383,-0.203485,-1.037867,0.560643,0.450349,fff676d8522543716287e474ed6eebe50db360ab


In [161]:
pd.read_csv('Data/users.csv')

,visitor_token_id,job,params
0,0000b08340cc86c7a61cc3251fbf2544c6472dda,бариста наставник,NaN
1,0004214be6142004e5dfae0c869f7648c33e3640,менеджер по работе с ключевыми клиентами,NaN
2,00099e4722876bce6828a4d083f59d9bc80284d1,NaN,ресторан кафе продукты питания и напитки
3,000aa3c29eee38748aa67148ec80f8754f27017f,NaN,кулинарная студия
4,000cf2d412eb10cae69abe61b606e8be3063d9ba,NaN,ресторан кафе ресторан кафе
...,...,...,...
38644,fff47c137b2ba6a66e4670fe3701f3088682b86e,су шеф кондитер,городской отель изучение рынка знакомство с но...
38645,fff5ecf120366e7a9982e7e01666f84280478336,NaN,NaN
38646,fff6302be5388f9160e86cb11a6be2cf487a1c5b,founder,кофейня общение с коллегами установление новых...
38647,fff676d8522543716287e474ed6eebe50db360ab,генеральный директор,изучение рынка знакомство с новинками получени...


In [160]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from sklearn.decomposition import PCA
from sklearn.base import TransformerMixin, BaseEstimator
from tqdm import tqdm


class BertTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, model_name="sberbank-ai/ruBert-base", batch_size=32):
        self.model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        self.model = AutoModel.from_pretrained(self.model_name)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)
        self.batch_size = batch_size

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        self.model.eval()
        embeddings = []

        for i in tqdm(range(0, len(X), self.batch_size)):
            batch = X[i:i + self.batch_size].to_numpy()
            inputs = self.tokenizer(list(batch), return_tensors="pt", truncation=True, padding=True, max_length=128)
            inputs = {key: value.to(self.device) for key, value in inputs.items()}

            with torch.no_grad():
                outputs = self.model(**inputs)
                cls_embeddings = outputs.last_hidden_state[:, 0, :]

            embeddings.extend(cls_embeddings.cpu().numpy())

        embeddings_array = np.array(embeddings)
        pca = PCA(n_components=10)
        reduced_embeddings = pca.fit_transform(embeddings_array)
        return reduced_embeddings

    def get_feature_names_out(self, input_features=None):
        return ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']


bert_t = [
    ('job', BertTransformer(), 'job'),
    ('par', BertTransformer(), 'params'),
]

bert_col_transform = ColumnTransformer(transformers=bert_t,
                                       remainder='passthrough',
                                       force_int_remainder_cols=False)

In [1]:
bert_data = bert_col_transform.fit_transform(user_data)

bert_cols = pd.Series(bert_col_transform.get_feature_names_out()).apply(lambda x: x.replace('__', '_'))
bert_cols[20] = 'visitor_token_id'

bert_data = pd.DataFrame(bert_data, columns=bert_cols)
bert_data.to_csv('Data/users_bert.csv', index=False)
bert_data


KeyboardInterrupt

